In [1]:
import uproot
import h5py
import numpy as np
import os
import re
import awkward as awk
import heplot as hepl
import matplotlib.pyplot as plt
import importlib
import glob
from heplot import plotting as heplt
import sys
sys.path.append('..')

In [2]:
import src.selection as sele
import src.string_constants as stco
import src.generator as gene
import src.reader as read

# read all files for each ds_id

In [3]:
ds_id = stco.ds_ids_sig['mumu'][0]

In [4]:
ds_id

'506196'

In [5]:
path = glob.glob(os.path.join(stco.in_dir_mc,'*'+ds_id+'*'))[0]

In [6]:
path

'/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.506196.MGPy8EG_Zmumu_FxFx_3jets_HT2bias_BF.D_FTAG2.e8382_s3681_r13144_p5981.GN2v01_Nom_output.root'

In [6]:
fnames = [os.path.join(path, ff) for ff in os.listdir(path)]

In [7]:
fnames

['/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.506196.MGPy8EG_Zmumu_FxFx_3jets_HT2bias_BF.D_FTAG2.e8382_s3681_r13144_p5981.GN2v01_Nom_output.root/user.ltoffoli.37305917._000001.output.root',
 '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.506196.MGPy8EG_Zmumu_FxFx_3jets_HT2bias_BF.D_FTAG2.e8382_s3681_r13144_p5981.GN2v01_Nom_output.root/user.ltoffoli.37305917._000002.output.root',
 '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.506196.MGPy8EG_Zmumu_FxFx_3jets_HT2bias_BF.D_FTAG2.e8382_s3681_r13144_p5981.GN2v01_Nom_output.root/user.ltoffoli.37305917._000003.output.root',
 '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.506196.MGPy8EG_Zmumu_FxFx_3jets_HT2bias_BF.D_FTAG2.e8382_s3681_r13144_p5981.GN2v01_Nom_output.root/user.

In [8]:
rt = uproot.open(fnames[0])

In [9]:
rt.classnames()

{'ee_2017;1': 'TDirectory',
 'ee_2017/cutflow;1': 'TH1D',
 'ee_2017/cutflow_mc;1': 'TH1D',
 'ee_2017/cutflow_mc_pu;1': 'TH1D',
 'ee_2017/cutflow_pu;1': 'TH1D',
 'ee_2017/cutflow_scale_factors;1': 'TH1D',
 'mumu_2017;1': 'TDirectory',
 'mumu_2017/cutflow;1': 'TH1D',
 'mumu_2017/cutflow_mc;1': 'TH1D',
 'mumu_2017/cutflow_mc_pu;1': 'TH1D',
 'mumu_2017/cutflow_pu;1': 'TH1D',
 'mumu_2017/cutflow_scale_factors;1': 'TH1D',
 'AnalysisTracking;1': 'TTree',
 'nominal;1': 'TTree',
 'sumWeights;1': 'TTree',
 'truth;1': 'TTree'}

In [10]:
np.array(rt['sumWeights/totalEventsWeighted'], dtype=np.float64)

array([5.84244864e+08, 5.53689216e+08, 5.50758528e+08, 5.13354688e+08,
       5.34187584e+08, 5.44187712e+08, 5.31054592e+08, 5.24199808e+08,
       5.07864352e+08, 5.22198976e+08, 5.29186240e+08, 5.29710368e+08,
       5.03878432e+08, 5.18727168e+08, 5.14299264e+08, 5.35412544e+08,
       5.18531072e+08, 5.14962080e+08, 5.09364096e+08, 5.25545728e+08,
       5.33605088e+08, 5.25215040e+08, 5.28455520e+08, 5.29186560e+08,
       5.23900992e+08, 5.19676320e+08, 5.04668160e+08, 4.56032000e+08,
       3.96412384e+08, 3.84934592e+08, 3.29302496e+08, 3.29336224e+08,
       3.23438944e+08, 3.24988000e+08, 3.29857344e+08, 3.24125472e+08,
       3.30185920e+08, 3.31909504e+08, 3.18836640e+08, 3.23254048e+08,
       3.26412416e+08, 3.28617024e+08, 3.22664512e+08, 3.30397344e+08,
       3.27347584e+08, 3.39302720e+08, 3.28087296e+08, 3.33126432e+08,
       3.24863712e+08, 3.30217664e+08])

In [11]:
evt_weights_per_file = np.array(rt['sumWeights/totalEventsWeighted'], dtype=np.float64).sum()

In [12]:
evt_weights_per_file

22023715264.0

# compute sum of weights value from all files belonging to a dsid

In [13]:
sow_dd = {}

In [14]:
for dsid in stco.ds_ids_all:
    dsid_path = glob.glob(os.path.join(stco.in_dir_mc,'*'+dsid+'*'))[0]
    fnames = [os.path.join(dsid_path, ff) for ff in os.listdir(dsid_path)]
    
    sow_per_dsid = 0
    for ff in fnames: # for all files
        rt = uproot.open(ff)
        sow_per_dsid += np.array(rt['sumWeights/totalEventsWeighted'], dtype=np.float64).sum() # for all branches in 
    
    sow_dd[dsid] = sow_per_dsid

In [15]:
sow_dd

{'410472': 101069830504.0,
 '363356': 6949688.5625,
 '363358': 2021873.654296875,
 '364302': 638141.81640625,
 '506193': 72501813184.0,
 '506194': 532006530368.0,
 '506195': 2068729490176.0,
 '506196': 83903746624.0,
 '506197': 438336549632.0,
 '506198': 2138878821504.0,
 '512198': 60103329024.0,
 '512199': 522860189264.0,
 '512200': 2332324618176.0}

In [16]:
len(sow_dd)

13

### compute mc lumi as sow_j / (xsec_j * k_j) for each dsid k and correspoding sum of weights, mc cross section and k-scale factor

In [20]:
mc_lumi_dd = {dsid : stco.lumi * (stco.scale_factors[dsid]['xsec'] * stco.scale_factors[dsid]['k']) / sow_dd[dsid] for dsid in stco.ds_ids_all}

In [21]:
mc_lumi_dd

{'410472': 3.844961587385072e-05,
 '363356': 0.013852264247011286,
 '363358': 0.07531852125199107,
 '364302': 0.009075010893258743,
 '506193': 3.0177168210226702e-05,
 '506194': 2.2872998058092888e-05,
 '506195': 4.1671964576028617e-05,
 '506196': 2.5696088324419277e-05,
 '506197': 2.7733533396213347e-05,
 '506198': 4.031973964348253e-05,
 '512198': 3.6394928093359385e-05,
 '512199': 2.343666217014798e-05,
 '512200': 3.6926749614020025e-05}